#### Find your Spark installation

In [1]:
import findspark
findspark.init('C:\spark-3.0.0-bin-hadoop2.7')

#### import Spark SQL and start session

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("my_first_spark_app") \
    .getOrCreate()

#### Load data from file

In [5]:
df = spark.read.csv("data.csv",header=True)

In [6]:
#shows the data (like head())  - this is an action so forces evaluation of pervious commands
df.show()

+---+---+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+----+
|_c0| ID|                T1|            T1_AVE|            T1_AVW|            T1_SDW|                T2|            T2_AVE|            T2_AVW|            T2_SDW|                T3|            T3_AVE|            T3_AVW|            T3_SDW|              Tamb|          Tamb_AVE|          Tamb_AVW|           Tamb_SDW|Time|
+---+---+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+----+
|  0|  4|  43.8573662769241| 40.38169618

In [29]:
# Print the schema
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Relative_Compactness: string (nullable = true)
 |-- Surface_Area: string (nullable = true)
 |-- Wall_Area: string (nullable = true)
 |-- Roof_Area: string (nullable = true)
 |-- Overall_Height: string (nullable = true)
 |-- Orientation: string (nullable = true)
 |-- Glazing_Area: string (nullable = true)
 |-- Cooling_Load: string (nullable = true)



In [30]:
#add a new column
df = df.withColumn('Wall2', df.Wall_Area + 2)

### Columns are strings, but we need to convert to numbers

In [31]:
#import double type from spark sql
from pyspark.sql.types import FloatType

#convert one column
df = df.withColumn("Wall_Area", df["Wall_Area"].cast(FloatType()))

#convert all columns
for col_name in df.columns:
    df = df.withColumn(col_name, df[col_name].cast(FloatType()))

In [32]:
df.printSchema()

root
 |-- _c0: float (nullable = true)
 |-- Relative_Compactness: float (nullable = true)
 |-- Surface_Area: float (nullable = true)
 |-- Wall_Area: float (nullable = true)
 |-- Roof_Area: float (nullable = true)
 |-- Overall_Height: float (nullable = true)
 |-- Orientation: float (nullable = true)
 |-- Glazing_Area: float (nullable = true)
 |-- Cooling_Load: float (nullable = true)
 |-- Wall2: float (nullable = true)



In [33]:
df = df.dropna()

### some built-in functions

In [34]:
#select columns
df2 = df.select(df['Orientation'], df['Roof_area'])

In [35]:
#filter roof areas
df2 = df2.filter(df2['Roof_area'] > 200)

### Aggregating functions

In [36]:
#calculate mean roof area for each orientation
df2 = df2.groupBy("Orientation").agg({'Roof_area': 'mean'})

In [37]:
#only when we called show() - an action - will the previous transformations be executed
df2.show()

+-----------+--------------+
|Orientation|avg(Roof_area)|
+-----------+--------------+
|        5.0|         220.5|
|        2.0|         220.5|
|        3.0|         220.5|
|        4.0|         220.5|
+-----------+--------------+



### User-defined function

In [38]:
from pyspark.sql.functions import udf

#define a python function
def square_float(x):
    return float(x**2)

#register a spark udf to use python function
square_udf = udf(lambda z: square_float(z), FloatType())

In [39]:
# create new column 'wall_area2 with result of udf
df3 = df.withColumn("Wall_Area2", square_udf(df.Wall_Area))

In [40]:
df3.show()

+----+--------------------+------------+---------+---------+--------------+-----------+------------+------------+-----+----------+
| _c0|Relative_Compactness|Surface_Area|Wall_Area|Roof_Area|Overall_Height|Orientation|Glazing_Area|Cooling_Load|Wall2|Wall_Area2|
+----+--------------------+------------+---------+---------+--------------+-----------+------------+------------+-----+----------+
| 1.0|                0.98|       514.5|    294.0|   110.25|           7.0|        2.0|         0.0|       21.33|296.0|   86436.0|
| 2.0|                0.98|       514.5|    294.0|   110.25|           7.0|        3.0|         0.0|       21.33|296.0|   86436.0|
| 3.0|                0.98|       514.5|    294.0|   110.25|           7.0|        4.0|         0.0|       21.33|296.0|   86436.0|
| 4.0|                0.98|       514.5|    294.0|   110.25|           7.0|        5.0|         0.0|       21.33|296.0|   86436.0|
| 5.0|                 0.9|       563.5|    318.5|    122.5|           7.0|        

### SQL

In [41]:
#register spark df as SQL temporary view
df.createOrReplaceTempView("energy")

In [42]:
#execute a SQL query
df_SQL = spark.sql("SELECT * FROM energy WHERE Roof_area > 200")

In [43]:
#show result - only now will the SQL query be executed
df_SQL.show()

+----+--------------------+------------+---------+---------+--------------+-----------+------------+------------+-----+
| _c0|Relative_Compactness|Surface_Area|Wall_Area|Roof_Area|Overall_Height|Orientation|Glazing_Area|Cooling_Load|Wall2|
+----+--------------------+------------+---------+---------+--------------+-----------+------------+------------+-----+
|25.0|                0.74|       686.0|    245.0|    220.5|           3.5|        2.0|         0.0|        10.9|247.0|
|26.0|                0.74|       686.0|    245.0|    220.5|           3.5|        3.0|         0.0|       11.19|247.0|
|27.0|                0.74|       686.0|    245.0|    220.5|           3.5|        4.0|         0.0|       10.94|247.0|
|28.0|                0.74|       686.0|    245.0|    220.5|           3.5|        5.0|         0.0|       11.17|247.0|
|29.0|                0.71|       710.5|    269.5|    220.5|           3.5|        2.0|         0.0|       11.27|271.5|
|30.0|                0.71|       710.5|

### Now we convert the result to a pandas dataframe. This action cause all the previous transformations (which have not yet been executed) to run beofre outputting the new pandas dataframe

In [44]:
df_pd = df2.toPandas()

### Alternately we could output the reuslt to a csv, this is also an action and will cause our transformation to be executed

In [45]:
#this will write each partition of the data into a single csv ina folder called 'energy_data2'
df2.write.csv("energy_data2")

In [46]:
#this will combine all partitioninto one file, and write a single csv in the folder call 'energy_data2_single'
#beware if you are writing a large file - 
# all the data willbe sent to a single executor and might crash your spark sessiom

df2.coalesce(1).write.csv("energy_data2_single")

### END

In [47]:
spark.stop()